# KoGPT2 모델을 이용하여 보다 자연스러운 챗봇 만들어보기

## 1. 데이터셋 가져오기

In [4]:
import pandas as pd
import re
import urllib.request

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1-1. Chatbot_data
출처 : https://github.com/songys/Chatbot_data

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
cbdata_df = pd.read_csv('ChatBotData.csv')
cbdata_df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


### 1-2. 트위터에서 수집 및 정제한 대화 시나리오

출처 : [트위터에서 수집 및 정제한 대화 시나리오 - AI Hub](https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-008)

In [ ]:
tt_df = pd.read_excel('/content/drive/MyDrive/cd_ai/s4p/twitter_talk.xlsx')
tt_df.head()

In [ ]:
tt_df = tt_df.dropna(axis='columns', how="all")
tt_df.head()

In [9]:
len(tt_df.columns)

83

In [10]:
[tt_df.iloc[0, 0], tt_df.iloc[0, 1]]

['기계식 키보드 써보고 싶어요.', '싼 것도 많던데 써보세요. 정말 좋아요.']

In [11]:
columns = ['Q', 'A', 'label']
twitter_talk = pd.DataFrame(columns=columns)

for row in range(len(tt_df.index)):
  if row % 500 == 0: print(f'row : {row}')
  for col in range(len(tt_df.columns)):
    if col+1 == len(tt_df.columns):
      break
    if pd.isna(tt_df.iloc[row, col+1]):
      break
    sent_couple = pd.DataFrame([[tt_df.iloc[row, col], tt_df.iloc[row, col+1], 0]], columns=columns)
    twitter_talk = twitter_talk.append(sent_couple, ignore_index=True)

row : 0
row : 500
row : 1000
row : 1500


In [12]:
twitter_talk.head()

,Q,A,label
0,기계식 키보드 써보고 싶어요.,싼 것도 많던데 써보세요. 정말 좋아요.,0
1,싼 것도 많던데 써보세요. 정말 좋아요.,싼 것도 있나요? 예전에 봤을 땐 다 10만 원이 넘던데요.,0
2,싼 것도 있나요? 예전에 봤을 땐 다 10만 원이 넘던데요.,이거 쓰는데 예쁘고 딸깍거려요.,0
3,이거 쓰는데 예쁘고 딸깍거려요.,우와 진짜 예뻐요.,0
4,우와 진짜 예뻐요.,불 끄면 더 예뻐요.,0


### 1-3. 한국어 감정 정보가 포함된 연속적 대화 데이터셋

출처 : [한국어 감정 정보가 포함된 연속적 대화 데이터셋 - AI Hub](https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-010)

In [13]:
ket_df = pd.read_excel('/content/drive/MyDrive/cd_ai/s4p/kor_emo_talk.xlsx')
ket_df.head()

,dialog #,발화,감정
0,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
1,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
2,NaN,손님 왔어요.,중립
3,NaN,손님? 누구?,중립
4,NaN,몰라요. 팀장님 친구래요.,중립


In [14]:
[ket_df.iloc[0, 1], ket_df.iloc[1, 1]]

['아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!',
 '그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.']

In [15]:
ket_df['감정'].unique()

array(['분노', '혐오', '중립', '놀람', '행복', '공포', '슬픔', 'ㅈ중립', '분ㄴ', '중림', nan,
       'ㅍ', 'ㄴ중립', '분', '줄'], dtype=object)

In [16]:
ket_df[(ket_df['감정'] == 'ㅈ중립') | (ket_df['감정'] == '중림') | (ket_df['감정'] == '중림') | (ket_df['감정'] == 'ㄴ중립') | (ket_df['감정'] == '줄')] = '중립'
ket_df[(ket_df['감정'] == '분ㄴ') | (ket_df['감정'] == '분')] = '분노'
ket_df[ket_df['감정'] == 'ㅍ'] = '공포'

In [17]:
ket_df[ket_df['감정'].isna()] = '중립' # NaN을 없애버리면 맥락을 잃을 수 있음. 대신 중립으로 설정

In [18]:
ket_df['감정'].unique()

array(['분노', '혐오', '중립', '놀람', '행복', '공포', '슬픔'], dtype=object)

In [19]:
ket_df[ket_df['감정'] == '놀람']

,dialog #,발화,감정
16,NaN,나? ... 나보고 하라고?,놀람
18,NaN,근데 왜... 나한테...?,놀람
48,NaN,... 오.,놀람
59,NaN,... 대리석?,놀람
105,NaN,요기잖아. 요기. 강 건너 윗동네도 몰라?,놀람
...,...,...,...
55464,NaN,...그거 괜찮긴 한 거야?,놀람
55523,NaN,……이 인형은 뭐야?,놀람
55585,NaN,어? 정말요?,놀람
55587,NaN,"혹시, 다들 은행 계좌없는 거예요?",놀람


In [20]:
# 중립 (0), 부정 (1), 긍정 (2)
neg_emotion = ['분노', '혐오', '공포', '슬픔']
pos_emotion = ['행복']
neutral_emotion = ['놀람', '중립'] # 놀람은 긍정 또는 부정 한쪽으로 구분하기 어려운 관계로 중립에 포함

ket_df['감정'][ket_df['감정'].isin(neg_emotion)] = 1
ket_df['감정'][ket_df['감정'].isin(pos_emotion)] = 2
ket_df['감정'][ket_df['감정'].isin(neutral_emotion)] = 0

In [21]:
ket_df.iloc[0,0] = 'start' # 시작 지점 설정(이후 동일한 열에서 새로운 맥락이 시작되는 지점은 S로 표시되어 있음)

In [22]:
ket_df.head()

,dialog #,발화,감정
0,start,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,1
1,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,1
2,NaN,손님 왔어요.,0
3,NaN,손님? 누구?,0
4,NaN,몰라요. 팀장님 친구래요.,0


In [23]:
columns = ['Q', 'A', 'label']
kor_emo_talk = pd.DataFrame(columns=columns)

for row in range(len(ket_df.index)):
  try:
    if row % 5000 == 0: print(f'row : {row}')
    if ket_df.iloc[row+1, 0] == 'S' : continue # NaN이 아닌 값(새로운 맥락의 대화)이 나오면 다음 순으로 넘기기
    sent_couple = pd.DataFrame([[ket_df.iloc[row, 1], ket_df.iloc[row+1, 1], ket_df.iloc[row+1, 2]]], columns=columns)
    kor_emo_talk = kor_emo_talk.append(sent_couple, ignore_index=True)
  except:
    print('done')

row : 0
row : 5000
row : 10000
row : 15000
row : 20000
row : 25000
row : 30000
row : 35000
row : 40000
row : 45000
row : 50000
row : 55000
done


In [24]:
kor_emo_talk.head()

,Q,A,label
0,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,1
1,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,손님 왔어요.,0
2,손님 왔어요.,손님? 누구?,0
3,손님? 누구?,몰라요. 팀장님 친구래요.,0
4,몰라요. 팀장님 친구래요.,내 친구? 친구 누구?,0


In [ ]:
kor_emo_talk[['Q', 'A']] = kor_emo_talk[['Q', 'A']].applymap(lambda x: x.replace(u'\xa0', u' '))

### 1-4. 한국어 대화 데이터셋

출처 : [한국어 대화 데이터셋 - AI Hub](https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-011)

In [25]:
# 응급 상황 대화
te_df = pd.read_excel('/content/drive/MyDrive/cd_ai/s4p/talk_emergency.xlsx')
te_df.head(25)

,num,talk
0,1,﻿아빠가 숨을 잘 못쉬어요.
1,2,그래요. 주소가 어떻게 되나요?
2,3,한남동 123번지 고마스빌딩이예요.
3,4,가슴에 통증이 있는지 물어봐줄래요?
4,5,있다고 해요. 이런경우는 처음이래요.
5,6,잠시만 기다려주세요. 도와줄 사람들을 금방 보내줄게요.알겠죠?
6,7,알겠어요. 빨리 오셔야 되요.
7,8,지금 사람들이 가고 있어요. 아버지는 지금 깨어있나요?
8,9,네
9,10,전에도 이런적이 있었는지 아빠한테 물어봐줄래요?


In [26]:
te_df.iloc[0,0] = 'start'
te_df.head()

,num,talk
0,start,﻿아빠가 숨을 잘 못쉬어요.
1,2,그래요. 주소가 어떻게 되나요?
2,3,한남동 123번지 고마스빌딩이예요.
3,4,가슴에 통증이 있는지 물어봐줄래요?
4,5,있다고 해요. 이런경우는 처음이래요.


In [27]:
columns = ['Q', 'A', 'label']
talk_emergency = pd.DataFrame(columns=columns)

for row in range(len(te_df.index)):
  try:
    if row % 1000 == 0: print(f'row : {row}')
    if te_df.iloc[row+1, 0] == 1 : continue
    sent_couple = pd.DataFrame([[te_df.iloc[row, 1], te_df.iloc[row+1, 1], 0]], columns=columns)
    talk_emergency = talk_emergency.append(sent_couple, ignore_index=True)
  except:
    print('done')

row : 0
row : 1000
row : 2000
row : 3000
row : 4000
done


In [30]:
talk_emergency.head()

,Q,A,label
0,﻿아빠가 숨을 잘 못쉬어요.,그래요. 주소가 어떻게 되나요?,0
1,그래요. 주소가 어떻게 되나요?,한남동 123번지 고마스빌딩이예요.,0
2,한남동 123번지 고마스빌딩이예요.,가슴에 통증이 있는지 물어봐줄래요?,0
3,가슴에 통증이 있는지 물어봐줄래요?,있다고 해요. 이런경우는 처음이래요.,0
4,있다고 해요. 이런경우는 처음이래요.,잠시만 기다려주세요. 도와줄 사람들을 금방 보내줄게요.알겠죠?,0


In [28]:
# 일상(회사) 대화
to_df = pd.read_excel('/content/drive/MyDrive/cd_ai/s4p/talk_office.xlsx')
to_df.head()

,num,talk
0,1,﻿좋은 아침.
1,2,안녕하세요.
2,1,좋은 아침.
3,2,반갑습니다.
4,1,좋은 아침.


In [29]:
columns = ['Q', 'A', 'label']
talk_office = pd.DataFrame(columns=columns)

for row in range(len(to_df.index)):
  try:
    if row % 1000 == 0: print(f'row : {row}')
    if row % 2 == 0:
      sent_couple = pd.DataFrame([[to_df.iloc[row, 1], to_df.iloc[row+1, 1], 0]], columns=columns)
      talk_office = talk_office.append(sent_couple, ignore_index=True)
  except:
    print('done')

row : 0
row : 1000
row : 2000


In [31]:
talk_office.head()

,Q,A,label
0,﻿좋은 아침.,안녕하세요.,0
1,좋은 아침.,반갑습니다.,0
2,좋은 아침.,좋은 아침이에요.,0
3,좋은 아침.,간밤에 별일 없으셨죠?,0
4,좋은 아침.,안녕하시렵니까?,0


### 1-5. 데이터셋 합치기

In [32]:
df = pd.concat([cbdata_df, twitter_talk, kor_emo_talk, talk_emergency, talk_office], ignore_index=True)

In [33]:
df.shape

(74588, 3)

In [36]:
df.to_csv('data_total.csv', encoding='utf-8')

## 2, 모델 학습시키기

In [37]:
# GPU 정보 
!nvidia-smi

Wed Jan 12 05:26:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [38]:
# KoGPT2-chatbot 소스 코드 복사
!git clone --recurse-submodules https://github.com/seahahn/KoGPT2-chatbot.git

Cloning into 'KoGPT2-chatbot'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 103 (delta 8), reused 8 (delta 3), pack-reused 85
Receiving objects: 100% (103/103), 115.98 KiB | 5.52 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Submodule 'Chatbot_data' (https://github.com/haven-jeon/Chatbot_data.git) registered for path 'Chatbot_data'
Cloning into '/content/KoGPT2-chatbot/Chatbot_data'...
remote: Enumerating objects: 24, done.        
remote: Counting objects: 100% (4/4), done.        
remote: Compressing objects: 100% (4/4), done.        
remote: Total 24 (delta 0), reused 3 (delta 0), pack-reused 20        
Submodule path 'Chatbot_data': checked out '235fac5aea3badab22743f7048afe936cf72f822'


In [39]:
# 폴더 이동
%cd KoGPT2-chatbot

/content/KoGPT2-chatbot


In [40]:
!pip install -r requirements.txt

     |████████████████████████████████| 841 kB 5.0 MB/s 
     |████████████████████████████████| 2.1 MB 46.3 MB/s 
     |████████████████████████████████| 596 kB 53.7 MB/s 
     |████████████████████████████████| 176 kB 57.1 MB/s 
     |████████████████████████████████| 829 kB 51.5 MB/s 
     |████████████████████████████████| 133 kB 59.9 MB/s 
     |████████████████████████████████| 895 kB 39.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.5 MB/s 
     |████████████████████████████████| 1.1 MB 61.3 MB/s 
     |████████████████████████████████| 160 kB 57.4 MB/s 
     |████████████████████████████████| 192 kB 63.3 MB/s 
     |████████████████████████████████| 271 kB 54.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=5fa24477220c933e7e7a4708eadf4bdebe61ac63364bc79d739a587289fdbd4b
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninsta

In [6]:
# 사전훈련된 KoGPT2를 챗봇 데이터로 파인튜닝
!CUDA_VISIBLE_DEVICES=0 python train_torch.py --train --gpus 1 --max_epochs 100 --batch-size 128 --data_path '/content/data_total.csv'

INFO:root:Namespace(accelerator=None, accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=128, benchmark=False, chat=False, check_val_every_n_epoch=1, checkpoint_callback=True, default_root_dir=None, deterministic=False, distributed_backend=None, enable_pl_optimizer=None, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=0, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=5e-05, max_epochs=100, max_len=32, max_steps=None, min_epochs=None, min_steps=None, model_params='model_chp/model_-last.ckpt', move_metrics_to_cpu=False, multiple_trainloader_mode='max_size_cycle', num_nodes=1, num_processes=1, num_sanity_val_steps=2, overfit_batches=0.0, plugins=None, precision=32, prepare_data_per_node=True, process_position=0, profiler=None, 

In [53]:
# 대화 테스트, `quit`를 입력하면 대화 종료
# 미리 학습해둔 가중치를 불러옴
# 만약 새로 학습한 가중치를 불러오려면 ckpt 파일의 경로를 해당 파일 경로로 지정해주면 됨
!CUDA_VISIBLE_DEVICES=0 python train_torch.py --gpus 1 --model_params '/content/drive/MyDrive/cd_ai/s4p/model_best.ckpt' --chat

INFO:root:Namespace(accelerator=None, accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=96, benchmark=False, chat=True, check_val_every_n_epoch=1, checkpoint_callback=True, data_path='Chatbot_data/ChatbotData.csv', default_root_dir=None, deterministic=False, distributed_backend=None, enable_pl_optimizer=None, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=0, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=5e-05, max_epochs=None, max_len=32, max_steps=None, min_epochs=None, min_steps=None, model_params='/content/drive/MyDrive/cd_ai/s4p/model_best.ckpt', move_metrics_to_cpu=False, multiple_trainloader_mode='max_size_cycle', num_nodes=1, num_processes=1, num_sanity_val_steps=2, overfit_batches=0.0, plugins=None, precision=32, 